# Dữ liệu ban đầu

In [3]:
import pandas as pd
import itertools as iter
pd.read_csv('store_data.csv',header = None)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,butter,light mayo,fresh bread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,burgers,frozen vegetables,eggs,french fries,magazines,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7499,escalope,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Preprocess

In [23]:
min_support = 0.0045 # suppose that frequent item was purchased 5 times/day => 1 week = 35
min_conf = 0.2
ms = 34

raw_data = pd.read_csv('store_data.csv',header = None)
records = [] # records = [ {} ]


for i in raw_data.index:
    records.append([str(raw_data.values[i][j]) for j in raw_data.columns if str(raw_data.values[i][j]) != 'nan'])
records = [set(i) for i in records]

K_1 = {} # K_1 = { () : {T_ID} }

# Create K_1
for i in range(0,len(records)):
    for j in records[i]:
        j = tuple((j,))
        if j not in K_1.keys():
            K_1[j] = {i,}
        else :
            K_1[j].add(i)

for x in list(K_1): # list(K_1): create a copy of the keys to be made
    if len(K_1[x]) < ms: 
        K_1.pop(x)
print(K_1)

{('almonds',): {0, 3588, 521, 6155, 6667, 5645, 528, 534, 2070, 3608, 544, 1570, 1574, 4649, 6697, 3116, 4142, 1584, 4145, 6195, 565, 4669, 6719, 4673, 6209, 6721, 1607, 2632, 6739, 7252, 3157, 1111, 3676, 94, 3166, 5217, 2148, 101, 102, 5734, 105, 5235, 118, 3711, 5759, 6275, 3716, 4741, 5252, 4236, 3213, 5778, 6813, 5280, 680, 6824, 3246, 7346, 691, 2739, 5814, 699, 707, 5831, 2255, 5839, 3283, 727, 1751, 730, 6876, 6877, 5854, 7398, 2794, 1774, 4337, 2803, 4852, 4348, 253, 1278, 3837, 4873, 5897, 6412, 781, 276, 788, 4374, 5396, 7444, 4378, 3869, 7453, 7454, 2850, 5922, 5930, 5421, 6449, 826, 4925, 3913, 1866, 3914, 2898, 4956, 7010, 4964, 6501, 2406, 872, 7019, 881, 1394, 4977, 1908, 2423, 3453, 2942, 3457, 6536, 1422, 2450, 6547, 5014, 1945, 1946, 3482, 414, 1442, 936, 1973, 3510, 952, 1468, 7105, 4042, 6091, 2510, 2513, 4050, 467, 6105, 3040, 3562, 4077, 4591, 4592, 6129, 6641, 1018}, ('shrimp',): {0, 6147, 6149, 4108, 6156, 6158, 16, 2064, 6163, 20, 23, 4120, 2078, 4127, 6175, 4

# Apriori algorithm creates all frequent items

In [38]:
# Create K-candidate(K >= 2) : cand{ sorted(items) : {T_ID} }
def candidates(items, k):
    cand = {} 
    tmp = [] # tmp[ {} ] : tránh duplicate
    for i,j in iter.combinations(items.keys(),2):
        inter = set(i).union(set(j)) 
        if (len(inter) == k) and (inter not in tmp):
            tmp.append(inter)
            inter = tuple(sorted((inter))) # sorted để sau này tìm cho dễ
            cand[inter] = items[i].intersection(items[j])
    return cand

# Create frequent-items sets
def frequent_items_set(candidates):
    cand = candidates
    for i in list(cand):
        if len(cand[i]) < ms:
            cand.pop(i)
    return cand

# Apriori algorithm:
def apriori():    
    global K_1
    fre_items = []
    fre_items.append(K_1) # fre_items[0] = K_1
    
    k = 0
    #print(fre_items[0])
    while len(fre_items[k]) > 0:
        fre_item = fre_items[k]
        candidate = candidates(fre_item,k+2)
        candidate = frequent_items_set(candidate)
        k += 1
        fre_items.append(candidate)
    return fre_items

freq_items = apriori()
print(freq_items[1])

{('almonds', 'mineral water'): {0, 3457, 3716, 521, 4236, 3213, 528, 6547, 788, 4374, 5014, 3608, 6813, 3869, 414, 544, 2850, 1442, 680, 4649, 6824, 5930, 4142, 1584, 4145, 6449, 691, 5814, 699, 4925, 6719, 6721, 1607, 2632, 3914, 2510, 2255, 2513, 2898, 727, 6105, 4956, 3166, 5854, 3040, 6501, 5734, 105, 3562, 4591, 4337, 881, 4977, 118, 3837, 2942, 3711}, ('almonds', 'green tea'): {0, 521, 5897, 4236, 5645, 3213, 5778, 2450, 7444, 5014, 544, 5280, 5922, 1574, 6449, 2739, 6209, 6721, 707, 1607, 1866, 3914, 6091, 2898, 727, 1751, 6105, 94, 3040, 5734, 2794, 3711, 1394, 1908, 2423, 4348, 3453, 5759}, ('almonds', 'burgers'): {6536, 4873, 3213, 781, 5778, 2450, 4374, 3482, 3869, 936, 5421, 4142, 3246, 6195, 691, 699, 4925, 6719, 4673, 5831, 3913, 1866, 2898, 3157, 5854, 94, 101, 102, 5734, 2406, 872, 1774, 881, 6129, 2803, 118, 2423, 4348, 3837}, ('almonds', 'eggs'): {3716, 4741, 6536, 6667, 1422, 276, 7444, 534, 2070, 4374, 3869, 5280, 1570, 1442, 6697, 1584, 4145, 7346, 6195, 691, 565, 

# Tạo luật

In [39]:
def create_righhand_side(right,k): # right : [ () ]
    right_items = []
    tmp = []
    for i,j in iter.combinations(right,2):
        items = set(i).union(set(j))
        if len(items) == k and items not in tmp:
            tmp.append(items)
            items = tuple(sorted(items))
            right_items.append(items)
    return right_items

def create_rules(freq_items):
    association_rules = []
    for freq_item in freq_items[1:len(freq_items)-1]:
        for freq_set in freq_item:
            subsets = [tuple((i,)) for i in freq_set] # right : 1 item 
            rule,right = caculate_conf(freq_items,freq_set,subsets)
            association_rules.extend(rule)
            k = 2
            while len(right) > 0  and len(right[0]) < len(freq_set)-1 :
                right = create_righhand_side(right,k)
                #print(right)
                rule,right = caculate_conf(freq_items,freq_set,right)
                #print(rule)
                association_rules.extend(rule)
                k += 1
    return association_rules
    

def caculate_conf(freq_items,supsets,subsets):
    rules = [] 
    right_items = [] #right
    for i in subsets: # duyệt mảng phần tử bên phải
        left_items = set(supsets).difference(set(i))
        left_items = tuple(sorted(left_items))
        confident_ratio = len(freq_items[len(supsets)-1][supsets]) / len(freq_items[len(left_items)-1][left_items]) 
        if confident_ratio >= min_conf: 
            lift = confident_ratio / len(freq_items[len(i)-1][i])*len(records)
            information = (left_items,i,confident_ratio,lift)
            rules.append(information)
            right_items.append(i)
    return rules,right_items

association_rules = create_rules(freq_items)
print(len(association_rules))
for i in association_rules:
    if i[3] >= 3:
        x = i[0]
        y = i[1]
        print("Rule: ",x,'--->',y)
        print("Confidence: ",i[2])
        print("Lift: ",i[3])
        print('=======================','\n')

700
Rule:  ('pasta',) ---> ('shrimp',)
Confidence:  0.3220338983050847
Lift:  4.506672147735896

Rule:  ('whole wheat pasta',) ---> ('olive oil',)
Confidence:  0.27149321266968324
Lift:  4.1224100976422955

Rule:  ('light cream',) ---> ('chicken',)
Confidence:  0.2905982905982906
Lift:  4.843950617283951

Rule:  ('pasta',) ---> ('escalope',)
Confidence:  0.3728813559322034
Lift:  4.700811850163794

Rule:  ('mushroom cream sauce',) ---> ('escalope',)
Confidence:  0.3006993006993007
Lift:  3.790832696715049

Rule:  ('herb & pepper',) ---> ('ground beef',)
Confidence:  0.32345013477088946
Lift:  3.2919938411349277

Rule:  ('tomato sauce',) ---> ('ground beef',)
Confidence:  0.37735849056603776
Lift:  3.8406594813240833

Rule:  ('mineral water', 'shrimp') ---> ('frozen vegetables',)
Confidence:  0.3050847457627119
Lift:  3.2006163328197226

Rule:  ('frozen vegetables', 'spaghetti') ---> ('shrimp',)
Confidence:  0.215311004784689
Lift:  3.0131489680782693

Rule:  ('chocolate', 'shrimp') ---